In [5]:
import json
import os
from collections import defaultdict
from pathlib import Path

data_directory = Path(os.path.abspath("")) / "data"

with open(data_directory / "stacked_lines.json") as f:
    stacked_lines = json.load(f)

date_groups = defaultdict(list)

for stacked_line in stacked_lines:
    date_groups[stacked_line['date']].append(stacked_line)

print({
    date: len(lines)
    for date, lines
    in date_groups.items()
})

{2457849.1443402776: 2, 2459039.171909722: 2, 2458030.3502430557: 1, 2453963.758414352: 4, 2453989.7788078706: 1}


In [6]:
from warm_pixels.fit.analysis import Analysis
from warm_pixels.fit.model import TrailModel
from warm_pixels import hst_utilities as ut, PixelLine
import autofit as af

# CCD
w = 84700.0
# Trap species
a = 0.17
b = 0.45
c = 0.38

analyses = []

beta = af.GaussianPrior(
    mean=0.478,
    sigma=0.1,
)

for date, pixel_lines in date_groups.items():
    # Trap lifetimes before or after the temperature change
    if date < ut.date_T_change:
        tau_a = 0.48
        tau_b = 4.86
        tau_c = 20.6
    else:
        tau_a = 0.74
        tau_b = 7.70
        tau_c = 37.0

    rho_q = af.UniformPrior(
        lower_limit=0.0,
        upper_limit=10.0,
    )

    model = af.Model(
        TrailModel,
        rho_q=rho_q,
        beta=beta,
        w=w,
        a=a,
        b=b,
        c=c,
        tau_a=tau_a,
        tau_b=tau_b,
        tau_c=tau_c,
    )
    model.add_assertion(c > 0.0)

    for pixel_line in pixel_lines:
        analysis = Analysis(
            pixel_line=PixelLine.from_dict(pixel_line),
        ).with_model(model)
        analyses.append(analysis)

analysis = sum(analyses)

In [7]:
"""
We make our optimiser which we can configure
"""

dynesty = af.DynestyStatic()

"""
Optimisation is performed by passing the model and the analysis to the fit function
"""

result = dynesty.fit(
    model=None,
    analysis=analysis,
)

print(type(result))
print(result)
print(result.model.rho_q)

2022-10-10 10:54:38,691 - autofit.non_linear.abstract_search - INFO - Creating search


AttributeError: 'NoneType' object has no attribute 'prior_count'